In [11]:
# coding=utf-8
__author__ = 'victor'

import os
import sys
sys.path.append('..')
sys.path.append('../../util')
reload(sys)
sys.setdefaultencoding('utf-8')

import db as dbcon
import pandas as pd
from datetime import datetime, timedelta

In [5]:
mongo = dbcon.connect_mongo()

In [24]:
def __evaluate_download(mongo, market, row, aspect='abs'):

    """
    evaluate artifact's download performance,
        abs, absolute download
        growth, 5 days' download delta rate
    """
    df = pd.DataFrame(list(mongo.trend.android.find({'apkname': row[1], 'appmarket': market,
                                                     'date': {'$gt': (datetime.today()-timedelta(days=15))}})))
    if df.shape[0] == 0:
        return 0
    df.fillna(method='pad')
    if aspect == 'abs':
        try:
            return df.download.mean()
        except AttributeError, ae:
            return -1
        except Exception, e:
            download = [int(x) for x in list(df.download)]
            return sum(download)/len(download)
    if aspect == 'growth':
        df = df.sort_values(by='date', ascending=False)
        return (df.iloc[0].download - df.iloc[-1].download) / df.shape[0]

In [25]:
row = [1, 'com.leho.mag.food']
market = 16010
__evaluate_download(mongo, market, row, 'growth')

0.0

In [23]:
df.sort_values(by='date')

,_id,apkname,appmarket,comment,date,download,score
6,5773e504e4861d428410e106,com.leho.mag.food,16010,0,2016-06-29,30000,3.6
5,57750688e4861d0f5d5470e9,com.leho.mag.food,16010,0,2016-06-30,30000,3.6
4,5775abf1e4861d3b23e08270,com.leho.mag.food,16010,0,2016-07-01,30000,3.6
3,57770459e4861d711adfaeee,com.leho.mag.food,16010,0,2016-07-02,30000,3.6
2,577857cae4861d1d99f8ffa3,com.leho.mag.food,16010,0,2016-07-03,30000,3.6
1,5779ac4be4861d7bd598bd04,com.leho.mag.food,16010,0,2016-07-04,30000,3.6
0,577afc12e4861d69738c016d,com.leho.mag.food,16010,0,2016-07-05,30000,3.6


In [10]:
f = lambda x: evaluate_download(mongo, 16020, x)
df['download'] = df.apply(f, 1)